## 1. Introduction
[PP-YOLO](https://arxiv.org/abs/2007.12099) is a optimized model based on YOLOv3 in PaddleDetection，whose performance(mAP on COCO) and inference spped are better than [YOLOv4](https://arxiv.org/abs/2004.10934). For more details, refer to [official documentation](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/configs/ppyolo/README.md).

## 2. Model Effects
PP-YOLO reached mmAP(IoU=0.5:0.95) as 45.9% on COCO test-dev2017 dataset, and inference speed of FP32 on single V100 is 72.9 FPS, inference speed of FP16 with TensorRT on single V100 is 155.6 FPS.

<div align="center">
  <img src="https://raw.githubusercontent.com/PaddlePaddle/PaddleDetection/release/2.5/docs/images/ppyolo_map_fps.png" width=500 />
</div>

## 3. How to use the model
Clone PaddleDetection firstly and put the COCO-style dataset in `dataset/coco`

In [ ]:
%mkdir -p ~/work
%cd ~/work/
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

%cd PaddleDetection
%mkdir -p demo_input demo_output
!pip install -r requirements.txt

### 3.1 Training
Training PP-YOLO on 8 GPUs with following command(all commands should be run under PaddleDetection dygraph directory as default)

In [ ]:
!python -m paddle.distributed.launch --log_dir=./ppyolo_dygraph/ --gpus 0,1,2,3,4,5,6,7 tools/train.py -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml

### 3.2 Inference
For inference deployment or benchmark, model exported with `tools/export_model.py` should be used and perform inference with Paddle inference library with following commands:

In [ ]:
# export model, model will be save in output/ppyolo as default
!wget -P demo_input -N https://paddledet.bj.bcebos.com/modelcenter/images/General/000000014439.jpg
!python tools/export_model.py -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml -o weights=https://paddledet.bj.bcebos.com/models/ppyolo_r50vd_dcn_1x_coco.pdparams

# inference with Paddle Inference library
!CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=output_inference/ppyolo_r50vd_dcn_1x_coco --image_file=demo_input/000000014439.jpg --device=GPU --output_dir=demo_output

## 4. Model principle
The overall archtecture of PP-YOLO is shown as follows:
<div align="center">
  <img src="https://paddledet.bj.bcebos.com/modelcenter/images/PP-YOLO-Arch.png" width=500 />
</div>

PP-YOLO improved performance and speed of YOLOv3 with following methods:

- Better backbone: ResNet50vd-DCN
- Larger training batch size: 8 GPUs and mini-batch size as 24 on each GPU
- [Drop Block](https://arxiv.org/abs/1810.12890)
- [Exponential Moving Average](https://www.investopedia.com/terms/e/ema.asp)
- [IoU Loss](https://arxiv.org/pdf/1902.09630.pdf)
- [Grid Sensitive](https://arxiv.org/abs/2004.10934)
- [Matrix NMS](https://arxiv.org/pdf/2003.10152.pdf)
- [CoordConv](https://arxiv.org/abs/1807.03247)
- [Spatial Pyramid Pooling](https://arxiv.org/abs/1406.4729)

For more details, please refer to our technical report：https://arxiv.org/abs/2007.12099

## 5. Attention
**All commands run on AI Studio's `jupyter` by default. If running on a terminal, remove the % or ! at the beginning of the command.**

## 6. Related papers and citations
```
@misc{long2020ppyolo,
title={PP-YOLO: An Effective and Efficient Implementation of Object Detector},
author={Xiang Long and Kaipeng Deng and Guanzhong Wang and Yang Zhang and Qingqing Dang and Yuan Gao and Hui Shen and Jianguo Ren and Shumin Han and Errui Ding and Shilei Wen},
year={2020},
eprint={2007.12099},
archivePrefix={arXiv},
primaryClass={cs.CV}
}
```